### Inspect Subtraction in Detail
Michael Wood-Vasey

2019-08-09

Some starting points to inspect DIA Processing

After working through this Notebook, a person should be able to
1. Load a set of coadd, science, difference image for a given visit, filter, tract+path.
2. Load a postage stamps of coadd, science, and difference images
3. Load the PSF object for an image and inspect the size of the PSF.
4. Subtract two PSFs from different images

Future Goals of this Notebook:
5. Understand PSF of each image and how the Image Difference software calculated the convolution between them.
6. Inspect convolution kernels in both ZOGY and A&L.

In [ ]:
import math
import os
import sys

import numpy as np

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
from lsst.daf.persistence import Butler
from lsst.geom import SpherePoint

In [ ]:
# The Postage Stamp routines are saved in a separate file here in the same folder as this Notebook
from image_stamp import make_cutout_image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Inspecting a set of subtractions provided by Bob Armstrong:
#desc-dc2-dia

2019-05-03

Bob:
"
I have processed a new batch of difference images for Run 1.2p.  This new batch creates templates from the  first two years that only uses visits with seeing < 0.7.  It then produces difference images for visits from the remaining years.  It combines all the filters except for u-band where there were not enough visits in the first two years.  This new run also produces forced photometry on the templates at the diaObject positions.  It uses the same default settings of Alard-Lupton for difference image as the run before.
"

In [ ]:
repo = '/global/cscratch1/sd/rearmstr/new_templates/diffim_template'

------

In [ ]:
butler = Butler(repo)

Here is the information necessary to find our sample images.  These are called "Data IDs" in the DM Science Pipeline language.

The template is fixed for the given repo.  Then we pick a good subtraction and a subtraction with clear ringing.

In [ ]:
tract, patch, filt = 4849, '6,6', 'r'
template_id = {'tract': tract, 'patch': patch, 'filter': filt}
good_id = {'visit': 1181556, 'raftName': 'R12', 'detector': 45, 'filter': filt}
ring_id = {'visit': 1203190, 'detector': 38, 'filter': filt}

Note that the calexps are from the central Run 1.2p processing, which is a parent of the repo used for these difference image tests.

In [ ]:
tmpl = butler.get(datasetType='deepCoadd', dataId=template_id)

new_good = butler.get(datasetType='calexp', dataId=good_id)
sub_good = butler.get(datasetType='deepDiff_differenceExp', dataId=good_id)

new_ring = butler.get(datasetType='calexp', dataId=ring_id)
sub_ring = butler.get(datasetType='deepDiff_differenceExp', dataId=ring_id)

In [ ]:
# Reading the diaSrc catalogs needs an updated DM Science Pipelines version to work.  
src_good = butler.get(datasetType='deepDiff_diaSrc', dataId=good_id)
src_ring = butler.get(datasetType='deepDiff_diaSrc', dataId=ring_id)

# The `src_good` and `src_ring` are the AFW Tables from the catalogs
# You might be more familar with them as AstroPy tables.
src_good = src_good.asAstropy()
src_ring = src_ring.asAstropy()

In [ ]:
sample_star = {'ra': 53.069214, 'dec': -28.343584}
sample_agn = {'ra': 53.135801, 'dec': -28.426165, 'diaObjectId': 21326977935867913}

In [ ]:
ra, dec = sample_agn['ra'], sample_agn['dec']

In [ ]:
title = 'Template: AGN'
agn_tmpl_cutout = make_cutout_image(butler, template_id, ra, dec, dataset_type='deepCoadd', title=title)

In [ ]:
title = 'Good Subtraction: AGN'
agn_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
title = 'Ringing Subtraction: AGN'
agn_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
ra, dec = sample_star['ra'], sample_star['dec']

In [ ]:
title = 'Template: Star'
star_tmpl_cutout = make_cutout_image(butler, template_id, ra, dec, dataset_type='deepCoadd', title=title)

In [ ]:
title = 'Good Subtraction: Star'
star_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

We can show the subtraction in the reference frame of the template, but note that this resamples the pixels.  I mostly present it here to show how to do it.

In the subtraction process itself, the opposite thing was done.  The template was warped to the science image.

In [ ]:
title = 'Good Subtraction: Star -- warped to template frame'
star_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title,
                                     warp_to_exposure=star_tmpl_cutout)

In [ ]:
title = 'Ringing Subtraction: Star'
star_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
title = 'Ringing Subtraction: Star - warped to template frame'
star_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title,
                                     warp_to_exposure=star_tmpl_cutout)

## PSF

Let's look at the PSFs of each image and see if we can predict the ringing seen above.

First a quick convenience function to translate shape to FWHM.

In [ ]:
def psf_fwhm(quad_shape):
    """FWHM in pixels"""
    xx, yy, xy = quad_shape.getIxx(), quad_shape.getIyy(), quad_shape.getIxy()
    fwhm_pixels = 2.355 * (xx * yy - xy * xy) ** 0.25
    return fwhm_pixels

And we're also going to want to look at the Fourier transform of this PSF so we'll write a function for that:

In [ ]:
from collections import namedtuple

In [ ]:
PSF_FFT = namedtuple('PSF_FFT', ['image', 'freq', 'fft', 'power'])

def show_fft(image):
    """Calculate and Show the FFT of the input array.
    
    Shows the 2D FFT and a 1D slice through x and y frequencies
    """
    (n, _) = np.shape(image)
    fft = np.fft.fft2(image)
    freq = np.fft.fftfreq(n)
    # Shift to center point
    fft = np.fft.fftshift(fft)
    freq = np.fft.fftshift(freq)
    power = np.abs(fft)**2

    _, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(power)
    axes[1].step(freq, power[:, n//2], where='mid', label='x')
    axes[1].step(freq, power[n//2, :], where='mid', label='y')
    axes[1].set_xlabel('Frequency [1/pixel]')
    axes[1].set_ylabel('Power')
    axes[1].legend()
        
    return PSF_FFT(image, freq, fft, power)

In [ ]:
radec = SpherePoint(ra, dec, afwGeom.degrees)

In [ ]:
tmpl_psf = tmpl.getPsf()

xy = afwGeom.PointD(tmpl.getWcs().skyToPixel(radec))

tmpl_quad_shape = tmpl_psf.computeShape(xy)
tmpl_kernel_image = tmpl_psf.computeKernelImage(xy)

print(tmpl_quad_shape)
print(f'FWHM: {psf_fwhm(tmpl_quad_shape)} pixels')

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(tmpl_kernel_image)
display.show_colorbar()

In [ ]:
new_good_psf = new_good.getPsf()
xy = afwGeom.PointD(new_good.getWcs().skyToPixel(radec))

new_good_quad_shape = new_good_psf.computeShape(xy)
new_good_kernel_image = new_good_psf.computeKernelImage(xy)

print(new_good_quad_shape)
print(f'FWHM: {psf_fwhm(new_good_quad_shape)} pixels')

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(new_good_kernel_image)
display.show_colorbar()

In [ ]:
new_ring_psf = new_ring.getPsf()

xy = afwGeom.PointD(new_ring.getWcs().skyToPixel(radec))

new_ring_quad_shape = new_ring_psf.computeShape(xy)
new_ring_kernel_image = new_ring_psf.computeKernelImage(xy)

print(new_ring_quad_shape)
print(f'FWHM: {psf_fwhm(new_ring_quad_shape)} pixels')

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(new_ring_kernel_image)
display.show_colorbar()

In [ ]:
tmpl_psf_fft = show_fft(tmpl_kernel_image.array)

Seems reasonable.  The Fourier Transform of a Gaussian is a Gaussian.  The PSF is largely Gaussian, and so is the FFT.

In [ ]:
new_good_psf_fft = show_fft(new_good_kernel_image.array)

In [ ]:
new_ring_psf_fft = show_fft(new_ring_kernel_image.array)

In [ ]:
def gaussian(x, mu=0, sigma=1, norm=None):
    """Return a normalized Gaussian evaluated along x"""
    if norm is None:
        norm = (1 / (np.sqrt(2*np.pi) * sigma))
    return norm * np.exp(-(np.asarray(x) - mu)**2/(2*sigma**2))

In [ ]:
def plot_slice_psf(psf_fft, psf_shape, label='', color=None):
    n, _ = np.shape(psf_fft.image)
    pixels = np.arange(-n//2 + 1, n//2 + 1)
    plt.step(pixels, psf_fft.image[:, n//2], where='mid',
             label=f'{label}_x', color=color, linestyle='-')
    plt.step(pixels, psf_fft.image[n//2, :], where='mid',
             label=f'{label}_y', color=color, linestyle='--')
    sigma_to_fwhm = 2 * math.sqrt(2 * math.log(2))
    fwhm_to_sigma = 1 / sigma_to_fwhm
    sigma = fwhm_to_sigma * psf_fwhm(psf_shape)
    smooth_pixels = np.linspace(-n//2 + 1, n//2 + 1, 101)
    plt.plot(smooth_pixels,
             gaussian(smooth_pixels, sigma=sigma, norm=psf_fft.image[n//2, n//2]),
             color=color, ls=':', label=fr'$\sigma={sigma:0.2f}$')

In [ ]:
plot_slice_psf(tmpl_psf_fft, tmpl_quad_shape, label='tmpl', color='blue')
plot_slice_psf(new_good_psf_fft, new_good_quad_shape, label='good', color='green')
plot_slice_psf(new_ring_psf_fft, new_ring_quad_shape, label='ring', color='orange')

plt.xticks(np.linspace(-10, +10, 21))
plt.xlim(-10, +10)
plt.legend();

The PSFs are normalized in 2D to a sum of 1.
They are not normalized to 1 in a 1D slices.

In [ ]:
print(np.sum(tmpl_kernel_image.array))
print(np.sum(new_good_kernel_image.array))
print(np.sum(new_ring_kernel_image.array))

In [ ]:
print(np.sum(tmpl_psf_fft.image[:, tmpl_n//2]))
print(np.sum(new_good_psf_fft.image[:, good_n//2]))
print(np.sum(new_ring_psf_fft.image[:, ring_n//2]))

------
Now let's show the difference between the PSF models

We above computed the realized PSF models as `tmpl_kernel_image`, `new_good_kernel_image`, `new_ring_kernel_image`.

By default the PSF images as created based on the PSF size.  So the image with a larger PSF will generate larger PSF stamps.  To aid comparison below we'll create images of the same size so that we can compare them.

In [ ]:
new_good_len = len(new_good_kernel_image.array)
tmpl_len = len(tmpl_kernel_image.array)
min_len = min(new_good_len, tmpl_len)

def get_center_box(im, size):
    nx, ny = im.array.shape
    bbox = afwGeom.Box2I(minimum=afwGeom.Point2I(x=-size//2+1, y=-size//2+1),
                         maximum=afwGeom.Point2I(x=+size//2, y=+size//2))
    return im[bbox]

new_good_center = get_center_box(new_good_kernel_image, min_len)
tmpl_center = get_center_box(tmpl_kernel_image, min_len)

# Renorm
new_good_center /= np.sum(new_good_center.array)
tmpl_center /= np.sum(tmpl_center.array)

from copy import copy, deepcopy
diff = deepcopy(new_good_center)
diff -= tmpl_center

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(diff)
display.show_colorbar('normalized counts')

In [ ]:
# Check normalizations
print(np.sum(tmpl_center.array))
print(np.sum(new_good_center.array))
print(np.sum(diff.array))

### Appendix
#### A. Just Get Me the Files!

While the following feature is not meant to be supported in the long-term (the Butler may eventually provide access to datasets across filesystems, remote, cloud buckets, etc.), for debugging and visualization it remains useful to get the direct filename.  We can get that using the Butler `getUri` function ("get Uniform Resource Identifier").

In [ ]:
template_file = butler.getUri(datasetType='deepCoadd', dataId=template_id)
good_new_file = butler.getUri(datasetType='calexp', dataId=good_id)
ring_new_file = butler.getUri(datasetType='calexp', dataId=ring_id)
good_sub_file = butler.getUri(datasetType='deepDiff_differenceExp', dataId=good_id)
ring_sub_file = butler.getUri(datasetType='deepDiff_differenceExp', dataId=ring_id)

If, for example you wanted to download these files to you local machine, you could do the following:
    
This isn't the recommended approach and doesn't scale to lots of subtractions, but I (MWV) found it useful when investigating things are our very early stages in understanding DC2 DIA.

In [ ]:
machine = 'cori.nersc.gov'
files_to_copy = [template_file, good_new_file, ring_new_file, good_sub_file, ring_sub_file]

for f in files_to_copy:
    print(f'rsync {machine}:{f} ./')